In [368]:
import numpy as np

In [369]:
# 데이터셋 생성
np.random.seed(4)

def generate_3d_data(m, w1=0.1, w2=0.3, noise=0.1):
    angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
    data = np.empty((m, 3))
    data[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
    data[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
    data[:, 2] = data[:, 0] * w1 + data[:, 1] * w2 + noise * np.random.randn(m)
    return data

X_train = generate_3d_data(60)
X_train = X_train - X_train.mean(axis=0, keepdims=0)

In [370]:
# 활성화 함수
class activation_function:
  # 시그모이드 함수
  def sigmoid(self, x):
    return 1 / (1+np.e**-x)
  
  # 시그모이드 함수의 미분 함수
  def sigmoid_diff(self, x):  
    return self.sigmoid(x) * (1 - self.sigmoid(x))

  # 항등 함수
  def equal(self, x):
    return x

  def equal_diff(self, x):
    return 1

In [371]:
# 비용 함수 클래스, 비용 함수의 종류에 따라 다른 클래스 내 함수를 사용한다.
class cost_function:
  # 예측값
  predict = []
  # 타겟값
  target = []
  # 비용 함수값
  error_cost = []

  # 오차 제곱합
  def errer_squared_sum(self, predict, target):
    self.predict = predict
    self.target = target

    self.error_cost = np.sum(0.5*((predict - target)**2))
    return self.error_cost

  # 오차 제곱합 미분 함수
  def diff_error_squared_sum(self, predict, target):
    self.predict = predict
    self.target = target

    return self.predict - self.target

In [372]:
class MLP:
  #가중치
  weight = []

  #편향값
  bias = []
    
  # 각 층별 노드들의 입력, 활성화 함수 연산 전의 값
  node_input = []

  # 각 층별 노드들의 출력
  node_output = []

  # 타겟, 목푯값
  target = []

  # 각 층의 델타 값의 저장
  delta = []

  # 비용 함수
  cost = cost_function()

  # 활성화 함수
  activation = activation_function()

  # 가중치 업데이트 크기
  weight_update_arr = []

  #순전파 계산
  def forward_cal(self, input, node_count):

    # 가중치의 임의 생성, node_count 개수만큼의 은닉(또는 출력) 노드가 존재한다.
    # (n,1) 개의 입력 값과 m 개의 노드 연결 (m, n) 크기의 가중치가 존재해야 한다.
    weight = np.random.rand(node_count, input.shape[0])

    #편향값의 임의 생성, 동일한 편향 값을 사용한다.
    bias = np.random.rand(1)

    # 노드 입력값의 저장 필요, 활성화 미분 함수의 값을 구하기 위해 필요
    self.node_input.append(input)

    #가중치와 노드 출력의 행렬곱연산, 편향값 덧셈
    hidden_input = weight @ input + bias
            
    #노드 입력과 활성화 함수 연산을 통한 노드 출력 계산
    # cnn 에선 0과 1 사이의 결과를 출력해야 하기 때문에 시그모이드 활성화 함수를 사용용
    output = self.activation.equal(hidden_input)

    # 노드 출력의 저장
    self.node_output.append(output)

    # 가중치 값 저장
    self.weight.append(weight)

    # 편향 값 저장
    self.bias.append(bias)

    return output

  # 반복을 수행하는 함수
  def forward_cal_iteration(self, input, target, learning_rate):
    # 가중치의 개수만큼 layer 가 쌓여있음
    for i in range(len(self.weight)):
      self.node_input.append(input)

      # 가중치와 노드 출력의 행렬곱 연산, 편향값 덧셈
      hidden_input = self.weight[i] @ input + self.bias[i]

      # 노드 출력 계산
      output = self.activation.equal(hidden_input)

      # 노드 출력의 저장
      self.node_output.append(output)

      input = output

    self.predict = output

    print(self.cost_function(target))

    self.cal_delta_result(self.predict, target)
    
    for i in range(len(self.weight) - 1):
      self.cal_delta_hidden()

    self.weight_update(learning_rate)
  
  def cost_function(self, target):
    self.target = target
    return self.cost.errer_squared_sum(self.predict, target)

  # 비용 함수에 대한 delta 값 계산
  def cal_delta_result(self, predict, target):  
    #delta 값 초기화
    self.delta = []

    #출력층 노드의 변화량에 대한 오차 함수의 변화량 계산
    delta = (self.cost.diff_error_squared_sum(predict, target) * self.activation.equal_diff(self.node_output[-1]))

    self.delta.append(delta)

    return delta

  # 은닉층에 대한 delta 값 계산, 가중치 변화량이 누적된다.
  def cal_delta_hidden(self):

    # delta 값 호출
    delta = self.delta[-1]

    # 가중치와의 연산
    delta = self.weight[-len(self.delta)].T @ delta

    # 활성화 함수 미분 함수와의 연산
    delta = delta * self.activation.sigmoid_diff(self.node_input[-(len(self.delta))])

    self.delta.append(delta)

    return delta
  
  # 가중치 업데이트량 크기 계산
  def weight_update(self, learning_rate):
    # 연산의 편리함을 위해 뒤집어준다.
    self.delta = self.delta[::-1]

    for i in range(len(self.weight)):
      self.weight_update_arr.append(self.delta[i] @ self.node_output[i].T)

      self.weight[i] = self.weight[i] - (self.weight_update_arr[i].T * learning_rate)

      self.bias[i] = self.bias[i] - (np.sum(self.delta[i]) * learning_rate)

In [373]:
X_train.shape

(60, 3)

In [374]:
mlp = MLP()

In [375]:
encoder = mlp.forward_cal(X_train[0], 2)
encoder

array([-0.90280681, -0.83461996])

In [376]:
decoder = mlp.forward_cal(encoder, 3)
decoder

array([-0.83932454, -0.58495808, -0.63274773])

In [377]:
mlp.weight

[array([[0.93994153, 0.80824973, 0.12321663],
        [0.90746221, 0.67706733, 0.3194252 ]]),
 array([[0.47496392, 0.56103256],
        [0.57182469, 0.15148925],
        [0.36198212, 0.43573474]])]

In [378]:
mlp.predict = decoder

In [379]:
mlp.cost_function(X_train[0])

0.0804105238979541

In [380]:
mlp.cal_delta_result(mlp.predict, X_train[0])

array([ 0.20044318,  0.17528038, -0.29986725])

In [381]:
mlp.cal_delta_hidden()

array([0.01783401, 0.00176144])

In [382]:
mlp.weight_update(0.01)

In [383]:
for i in range(10): 
  mlp.forward_cal_iteration(X_train[0], X_train[0], 0.01)

0.08020234241677547
0.08002112187476836
0.07986461913756647
0.07973075442978945
0.07961760009749111
0.07952337012188039
0.07944641033487779
0.07938518929029029
0.07933828974740413
0.07930440072661264
